## Experimentação de modelos
 Nesta sessão irei realizar uma série de experimentos, combinando diferentes modelos e preprocessamentos. Para registrar tudo isso, irei utilizar o MLFlow.

Aqui irei testar modelos baseados em distância e árvore, segue os esboços dos testes a serem realizados:

Baseados em estatística:
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.


Baseados em árvore:
- CatBoost Encoder nas categóricas
- CatBoost Encoder nas categóricas + Isolation Forest para rotular os outliers.

In [1]:
# Importando as bibliotecas

import category_encoders as ce
import mlflow
import numpy as np
import pandas as pd
import warnings

# Métodos para criar transformers
from sklearn.base import BaseEstimator, TransformerMixin

# Pré-processamento
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import IsolationForest

# Modelos
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Metricas
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Ignorando as mensagens de erro
warnings.filterwarnings('ignore')

In [2]:
class IFInput(BaseEstimator, TransformerMixin):
    def __init__(self, column, random_state = None):
    
        # Salvando o input localmente
        self.column = column
        self.random_state = random_state
    
    def fit(self, X, y=None):
        
        # Calculando os quartis e o IQR
        q1 = np.quantile(X[self.column], 0.25)
        q3 = np.quantile(X[self.column], 0.75)
        iqr = q3 - q1

        # Calculando o grau de contaminação
        qtd_outliers = X.query(f"{self.column} > {q3 + 1.5 * iqr}").shape[0]
        qtd_total_registros = X.shape[0]
        prcnt_contamination = qtd_outliers/qtd_total_registros
        
        # Salvando o modelo localmente
        self.iso_forest = IsolationForest(contamination=prcnt_contamination,
                                          random_state=self.random_state)
        
        # Fitando os dados
        self.iso_forest.fit(X[self.column].values.reshape(-1, 1))
        
        return self

    def transform(self, X, y=None):
        
        self.label = self.iso_forest.predict(X[self.column].values.reshape(-1, 1))
        
        # Adiciona os valores ao dataset
        X['is_outlier'] = self.label
        
        return X

In [3]:
def registrarexperimento(modelos, tags, transformer, x_treino, y_treino, IF = None):
    """
    Função para registrar experimentos de acordo com modelos, tags e transformadores específicos.
    :param modelos: lista contendo os modelos a serem testados
    :param tags: lista contendo tags de identificação para cada modelo e transformações aplicadas
    :param transformer: transformações que serão aplicadas ao conjunto de treino
    :param x_treino: dados contendo as variáveis dependentes
    :param y_treino: target
    :param IF: algorítmo para detectar outliers
    :return: None
    """
    # Iterando com os modelos da lista
    for model in modelos:

        # Iniciando os experimentos
        with mlflow.start_run():

            # Colocando uma tag para identificação
            index_modelo = modelos.index(model)
            mlflow.set_tag('modelo', tags[index_modelo])

            if IF != None:
            
                # Criando o pipeline
                pipe = Pipeline([('outlier_detect', IF),
                                 ('transformer', transformer),
                                 ('regressor', model)])

            else:
                
                # Criando o pipeline
                pipe = Pipeline([('transformer', transformer),
                                 ('regressor', model)])
                
            # Calculando as métricas com cross-validation
            mae_mean = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_mean_absolute_error').mean() * (-1)

            mae_std = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_mean_absolute_error').std() * (-1)

            rmse_mean = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_root_mean_squared_error').mean() * (-1)

            rmse_std = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_root_mean_squared_error').std() * (-1)

            # Salvando as métricas
            mlflow.log_metric('mae_mean', mae_mean)
            mlflow.log_metric('mae_std', mae_std)
            mlflow.log_metric('rmse_mean', rmse_mean)
            mlflow.log_metric('rmse_std', rmse_std)

    return None

In [4]:
# Importando os dados
dados = pd.read_csv('../data/interim/dados_para_treino.csv', index_col=0)

In [5]:
# Checando a tabela
dados.head()

,category,customer_type,unit_price,quantity,total,payment_type,lvl_estoque_past,lvl_estoque_to_predict,mean,median,min,max,std,var,day_of_week,is_weekend,hour,turn
0,fruit,gold,3.99,2,7.98,e-wallet,0.37,0.48,-0.6729,-1.26,-30.58,34.33,11.456305,131.246935,2,no,9,morning
1,fruit,premium,3.99,3,11.97,credit card,0.37,0.48,-0.6729,-1.26,-30.58,34.33,11.456305,131.246935,2,no,9,morning
2,fruit,premium,1.49,4,5.96,e-wallet,0.54,0.33,-0.6729,-1.26,-30.58,34.33,11.456305,131.246935,2,no,9,morning
3,fruit,non-member,3.99,3,11.97,cash,0.51,0.04,-0.6729,-1.26,-30.58,34.33,11.456305,131.246935,2,no,9,morning
4,fruit,premium,4.49,2,8.98,credit card,0.71,0.98,-0.6729,-1.26,-30.58,34.33,11.456305,131.246935,2,no,9,morning


### Alterando o tipo das colunas
Antes de iniciar os experimentos irei realizar uma pequena mudança nos tipos das colunas.

In [6]:
# Alterando o tipo de dado
dados['day_of_week'] = dados.day_of_week.astype('object')
dados['customer_type'] = dados.customer_type.astype('category')

## Buscando o melhor algorítmo

A partir daqui iniciarei os experimentos. Nesta etapa **NÃO** irei alterar os parâmetros, onde o melhor modelo irá receber tunning em outra rodada de experimentos.

Iniciarei os experimentos com os algoritmos **baseados em distância**.

In [7]:
# Define o local para salvar os exoerimentos
mlflow.set_tracking_uri('../mlruns')

# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

# Dividindo os dados em variáveis dependentes e independentes
x = dados.drop(columns='lvl_estoque_to_predict')
y = dados.lvl_estoque_to_predict

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,
                                                        y,
                                                        test_size=0.25,
                                                        random_state=14)

# Instanciando os modelos
linear_reg = LinearRegression()
lasso = Lasso(random_state=47)
ridge = Ridge(random_state=47)
elastic_nt = ElasticNet(random_state=47)
reg_estocastico = SGDRegressor(random_state=47)

# Criando listas com os modelo
modelos = [linear_reg, lasso, ridge, elastic_nt, reg_estocastico]

### Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas.

In [8]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).columns
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, numericas),
                                              ('encoder', ohe, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_OHE', 'Lasso_SC_OHE', 'Ridge_SC_OHE', 'Elastic_Net_SC_OHE', 'Reg_Estocástico_SC_OHE']

In [9]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais

In [10]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')
oe = ce.ordinal.OrdinalEncoder()

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).columns
categoricas = x_treino.select_dtypes('object').columns
ordinais = x_treino.select_dtypes('category').columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, numericas),
                                              ('encoder_nominal', ohe, categoricas),
                                              ('encoder_ordinal', oe, ordinais)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_OHE_OE', 'Lasso_SC_OHE_OE', 'Ridge_SC_OHE_OE', 'Elastic_Net_SC_OHE_OE', 'Reg_Estocástico_SC_OHE_OE']

In [11]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas.


In [12]:
# Criando uma lista vazia
col_com_outlier = []

# Buscando todas as variáveis numéricas
numericas = x_treino.select_dtypes(['int', 'float']).columns

# Iterando sobre as colunas numéricas
for col in numericas:

    # Calculando os quartis e o IQR
    q1 = np.quantile(dados[col], 0.25)
    q3 = np.quantile(dados[col], 0.75)
    iqr = q3 - q1

    #Separando os dados sem e com outliers
    if dados.query(f"{col} > {q3 + 1.5 * iqr}").shape[0] > 0:
        col_com_outlier.append(col)

In [13]:
# Instanciando os transformadores
sc = StandardScaler()
rs = RobustScaler()
ohe = OneHotEncoder(drop='first')

# Selecionando os dados por tipo
num_com_outliers = col_com_outlier
num_sem_outliers = [col for col in x_treino.select_dtypes(['int', 'float']).columns if col not in col_com_outlier]
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, num_sem_outliers),
                                              ('scaler_outliers', rs, num_com_outliers),
                                              ('encoder_nominal', ohe, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_RS_OHE', 'Lasso_SC_RS_OHE', 'Ridge_SC_RS_OHE', 'Elastic_Net_SC_RS_OHE', 'Reg_Estocástico_SC_RS_OHE']

In [14]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais

In [15]:
# Criando uma lista vazia
col_com_outlier = []

# Buscando todas as variáveis numéricas
numericas = x_treino.select_dtypes(['int', 'float']).columns

# Iterando sobre as colunas numéricas
for col in numericas:

    # Calculando os quartis e o IQR
    q1 = np.quantile(dados[col], 0.25)
    q3 = np.quantile(dados[col], 0.75)
    iqr = q3 - q1

    #Separando os dados sem e com outliers
    if dados.query(f"{col} > {q3 + 1.5 * iqr}").shape[0] > 0:
        col_com_outlier.append(col)

In [16]:
# Instanciando os transformadores
sc = StandardScaler()
rs = RobustScaler()
ohe = OneHotEncoder(drop='first')
oe = ce.ordinal.OrdinalEncoder()

# Selecionando os dados por tipo
num_com_outliers = col_com_outlier
num_sem_outliers = [col for col in x_treino.select_dtypes(['int', 'float']).columns if col not in col_com_outlier]
categoricas = x_treino.select_dtypes('object').columns
ordinais = x_treino.select_dtypes('category').columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, num_sem_outliers),
                                              ('scaler_outliers', rs, num_com_outliers),
                                              ('encoder_nominal', ohe, categoricas),
                                              ('encoder_ordinal', oe, ordinais)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_RS_OHE_OE', 'Lasso_SC_RS_OHE_OE', 'Ridge_SC_RS_OHE_OE', 'Elastic_Net_SC_RS_OHE_OE', 'Reg_Estocástico_SC_RS_OHE_OE']

In [17]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers

In [18]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')
outdetector = IFInput('total', random_state=47)

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).drop(columns='total').columns
categoricas = x_treino.select_dtypes(['object', 'category']).columns
outlier = ['total']

# Criando o transformer
transformer = ColumnTransformer(transformers=[('encoder', ohe, categoricas),
                                              ('scaler', sc, numericas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_OHE_IF', 'Lasso_SC_OHE_IF', 'Ridge_SC_OHE_IF', 'Elastic_Net_SC_OHE_IF', 'Reg_Estocástico_SC_OHE_IF']

In [19]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino, outdetector)

### Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers

In [20]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')
oe = ce.ordinal.OrdinalEncoder()
outdetector = IFInput('total', random_state=47)

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).columns
categoricas = x_treino.select_dtypes('object').columns
ordinais = x_treino.select_dtypes('category').columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, numericas),
                                              ('encoder_nominal', ohe, categoricas),
                                              ('encoder_ordinal', oe, ordinais)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_OHE_OE_IF', 'Lasso_SC_OHE_OE_IF', 'Ridge_SC_OHE_OE_IF', 'Elastic_Net_SC_OHE_OE_IF', 'Reg_Estocástico_SC_OHE_OE_IF']

In [21]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino, outdetector)

### Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers

In [22]:
# Criando uma lista vazia
col_com_outlier = []

# Buscando todas as variáveis numéricas
numericas = x_treino.select_dtypes(['int', 'float']).columns

# Iterando sobre as colunas numéricas
for col in numericas:

    # Calculando os quartis e o IQR
    q1 = np.quantile(dados[col], 0.25)
    q3 = np.quantile(dados[col], 0.75)
    iqr = q3 - q1

    #Separando os dados sem e com outliers
    if dados.query(f"{col} > {q3 + 1.5 * iqr}").shape[0] > 0:
        col_com_outlier.append(col)

In [23]:
# Instanciando os transformadores
sc = StandardScaler()
rs = RobustScaler()
ohe = OneHotEncoder(drop='first')
outdetector = IFInput('total', random_state=47)

# Selecionando os dados por tipo
num_com_outliers = col_com_outlier
num_sem_outliers = [col for col in x_treino.select_dtypes(['int', 'float']).columns if col not in col_com_outlier]
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, num_sem_outliers),
                                              ('scaler_outliers', rs, num_com_outliers),
                                              ('encoder_nominal', ohe, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_RS_OHE_IF', 'Lasso_SC_RS_OHE_IF', 'Ridge_SC_RS_OHE_IF', 'Elastic_Net_SC_RS_OHE_IF', 'Reg_Estocástico_SC_RS_OHE_IF']

In [24]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino, outdetector)

### Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers



In [25]:
# Criando uma lista vazia
col_com_outlier = []

# Buscando todas as variáveis numéricas
numericas = x_treino.select_dtypes(['int', 'float']).columns

# Iterando sobre as colunas numéricas
for col in numericas:

    # Calculando os quartis e o IQR
    q1 = np.quantile(dados[col], 0.25)
    q3 = np.quantile(dados[col], 0.75)
    iqr = q3 - q1

    #Separando os dados sem e com outliers
    if dados.query(f"{col} > {q3 + 1.5 * iqr}").shape[0] > 0:
        col_com_outlier.append(col)

In [26]:
# Instanciando os transformadores
sc = StandardScaler()
rs = RobustScaler()
ohe = OneHotEncoder(drop='first')
oe = ce.ordinal.OrdinalEncoder()
outdetector = IFInput('total', random_state=47)

# Selecionando os dados por tipo
num_com_outliers = col_com_outlier
num_sem_outliers = [col for col in x_treino.select_dtypes(['int', 'float']).columns if col not in col_com_outlier]
categoricas = x_treino.select_dtypes('object').columns
ordinais = x_treino.select_dtypes('category').columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, num_sem_outliers),
                                              ('scaler_outliers', rs, num_com_outliers),
                                              ('encoder_nominal', ohe, categoricas),
                                              ('encoder_ordinal', oe, ordinais)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_RS_OHE_OE_IF', 'Lasso_SC_RS_OHE_OE_IF', 'Ridge_SC_RS_OHE_OE_IF', 'Elastic_Net_SC_RS_OHE_OE_IF', 'Reg_Estocástico_SC_RS_OHE_OE_IF']

In [27]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino, outdetector)

A partir desse ponto iniciarei os experimentos com algoritmos **baseados em árvore**. Lembrando que algorítmos desse tipo não necessitam de um scaler, pois não são sensíveis as escalas dos dados. Além disso, aplicar One Hot Encoder não é recomendado, pois essa técnica irá aumentar a cardinalidade dos dados, prejudicando o modelo.

In [28]:
# Instanciando os modelos
tree_reg = DecisionTreeRegressor(random_state=47)
xtree_reg = ExtraTreeRegressor(random_state=47)
rf_reg = RandomForestRegressor(random_state=47)
gb_reg = GradientBoostingRegressor(random_state=47)
xgb_reg = XGBRegressor(random_state=47)

# Criando listas com os modelo
modelos = [tree_reg, xtree_reg, rf_reg, gb_reg, xgb_reg]

### CatBoost Encoder nas categóricas


In [29]:
# Instanciando os transformadores
cbenc = ce.CatBoostEncoder()

# Selecionando os dados por tipo
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('encoder', cbenc, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['DTree_reg_CtbEnc', 'XTree_reg_CtbEnc', 'RDF_reg_CtbEnc', 'GB_reg_CtbEnc', 'XGb_reg__CtbEnc']

In [30]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### CatBoost Encoder nas categóricas + Isolation Forest para rotular os outliers

In [ ]:
# Instanciando os transformadores
outdetector = IFInput('total', random_state=47)
cbenc = ce.CatBoostEncoder()

# Selecionando os dados por tipo
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('encoder', cbenc, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['DTree_reg_CtbEnc', 'XTree_reg_CtbEnc', 'RDF_reg_CtbEnc', 'GB_reg_CtbEnc', 'XGb_reg__CtbEnc']

In [ ]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino, outdetector)

c:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\site-packages\category_encoders\cat_boost.py:173: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  category = pd.Categorical(series)
c:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\site-packages\category_encoders\cat_boost.py:173: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  category = pd.Categorical(series)
c:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\site-packages\category_encoders\cat_boost.py:173: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  category = pd.Categorical(series)
c:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\site-packages\catego

IndexError: list index out of range